In [13]:
!pip install aimodelshare --upgrade -q
!pip install dabl
!pip install mljar-supervised

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
mljar-supervised 0.11.3 requires scikit-learn>=1.0, but you have scikit-learn 0.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563 kB 3.9 MB/s 
     |████████████████████████████████| 11.2 MB 37.5 MB/s 
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
     |████████████████████████████████| 965 kB 41.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: sc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import aimodelshare as ai
import dabl
import numpy as np
import pandas as pd

X_train, X_test, y_train, y_test, example_data, y_test_labels = ai.import_quickstart_data("titanic")

types = dabl.detect_types(X_train)
indices = list(types.index)
names = types.columns.to_numpy()
types = types.to_numpy()

d = dict()
for index, tpes in zip(indices, types):
  for name, tpe in zip(names, tpes):
    if tpe:
      d[index] = name
  
X_train_processed = dabl.clean(X_train, type_hints=d)
X_test_processed = dabl.clean(X_test, type_hints=d)

preprocessor = dabl.EasyPreprocessor()
preprocessor.fit(X_train_processed)
X_train_processed = preprocessor.transform(X_train_processed)
preprocessor.fit(X_test_processed)
X_test_processed = preprocessor.transform(X_test_processed)

y_train_labels_processed = np.asarray(list(map(lambda x: 0 if x == y_test_labels[0] else 1, y_train))).reshape(len(X_train), 1)
y_test_labels_processed = np.asarray(list(map(lambda x: 0 if x == y_test_labels[0] else 1, y_test)))

titanic = pd.DataFrame(np.concatenate((X_train_processed, y_train_labels_processed), axis=1), columns=["feature_" + str(i) for i in range(X_train_processed.shape[1])] + ["classification"])
ec = dabl.SimpleClassifier(random_state=0).fit(titanic, target_col="classification")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
Running DummyClassifier(random_state=0)
accuracy: 0.617 average_precision: 0.383 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.382
=== new best DummyClassifier(random_state=0) (using recall_macro):
accuracy: 0.617 average_precision: 0.383 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.382

Running GaussianNB()
accuracy: 0.775 average_precision: 0.780 roc_auc: 0.828 recall_macro: 0.770 f1_macro: 0.765
=== new best GaussianNB() (using recall_macro):
accuracy: 0.775 average_precision: 0.780 roc_auc: 0.828 recall_macro: 0.770 f1_macro: 0.765

Running MultinomialNB()
accuracy: 0.777 average_precision: 0.817 roc_auc: 0.836 recall_macro: 0.770 f1_macro: 0.766
=== new best MultinomialNB() (using recall_macro):
accuracy: 0.777 average_precision: 0.817 roc_auc: 0.836 recall_macro: 0.770 f1_macro: 0.766

Running DecisionTreeClass

Seems that dabl does not include xgboost, lightgbm...try another automl lib instead

Default algos: catboost, xgboost, random forest, lightgbm...

Automatically test all possible models by setting mode to 'Compete'

In [ ]:
import os
import shutil
automl_dir = "AutoML_classifier"
if os.path.exists(automl_dir):
    shutil.rmtree(automl_dir)

In [ ]:
import aimodelshare as ai
from supervised.automl import AutoML

X_train, X_test, y_train, y_test, example_data, y_test_labels = ai.import_quickstart_data("titanic")

# automl = AutoML(results_path=automl_dir, mode="Compete")
automl = AutoML(results_path=automl_dir)#, mode="Compete")
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
AutoML directory: AutoML_classifier
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline logloss 0.663015 trained in 1.17 seconds


2_DecisionTree logloss 0.519159 trained in 17.96 seconds
3_Linear logloss 0.524437 trained in 3.24 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost logloss 0.492153 trained in 5.04 seconds
5_Default_NeuralNetwork logloss 0.5878 trained in 1.6 seconds
6_Default_RandomForest logloss 0.527079 trained in 6.56 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.489899 trained in 0.58 seconds
AutoML fit time: 49.81 seconds
AutoML best model: Ensemble


In [ ]:
from sklearn.metrics import accuracy_score

print(f"Accuracy: {accuracy_score(y_test, predictions)*100.0:.2f}%" )

Accuracy: 80.15%


Save automl model into onnx model is not supported yet

In [ ]:
automl.get_leaderboard()

,name,model_type,metric_type,metric_value,train_time
0,1_Baseline,Baseline,logloss,0.663015,3.14
1,2_DecisionTree,Decision Tree,logloss,0.519159,19.33
2,3_Linear,Linear,logloss,0.524437,4.63
3,4_Default_Xgboost,Xgboost,logloss,0.492153,6.48
4,5_Default_NeuralNetwork,Neural Network,logloss,0.587800,2.84
5,6_Default_RandomForest,Random Forest,logloss,0.527079,8.01
6,Ensemble,Ensemble,logloss,0.489899,0.58


In [ ]:
selected_models = automl.ensemble.selected_models
models = []
for model_ in selected_models:
  model = model_["model"]
  models.append([model.get_name(), model])
models

[['2_DecisionTree',
 ['4_Default_Xgboost',

Get weights

In [ ]:
os.chdir(automl_dir)

In [ ]:
import glob

loaded_model = []
for model_name, model in models:
  os.chdir(model_name)
  
  model.load("./", "")
  os.chdir("..") 

  loaded_model.append(model)

In [ ]:
import numpy as np

for model in loaded_model:
  predictions = model.predict(X_test)
  predict_cols = predictions.columns
  predict_cols = np.asarray([name.split("_")[-1] for name in predict_cols])
  predictions = predict_cols[predictions.to_numpy().argmax(axis=-1)]
  print(f"Accuracy: {accuracy_score(y_test, predictions)*100.0:.2f}%" )

Accuracy: 79.77%
Accuracy: 77.10%


In [ ]:
model.learners

Not be able to transfer to onnx with best model: model is not a standard classifier

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacked_classifier = StackingClassifier(estimators=[
            ("model_"+str(i), model) for i, model in enumerate(loaded_model)], 
            final_estimator=LogisticRegression())

stacked_classifier.classes_ #?? not exist

pipeline = make_pipeline(stacked_classifier)
pipeline.score(X_test,y_test)


In [ ]:
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]])) for _ in range(len(loaded_model))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(pipeline, framework='sklearn',
              initial_types=initial_type,
              transfer_learning=False, deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())